## GoogLeNet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import googlenet

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# 데이터셋 불러오기 #812.8MB, 359장 이미지
train_dataset = datasets.ImageFolder(root='//Users/hairyungsung/Documents/workspaces/workspace_ds/train_face_0807', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 모델 정의
model = googlenet(pretrained=True)  # 이미 ImageNet 데이터셋으로 미리 학습된 가중치 사용
model.fc = nn.Linear(1024, 7)  # 새로운 fully connected layer를 추가하여 클래스 수에 맞게 출력 설정
model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 모델 학습
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print("Finished Training")


/opt/homebrew/anaconda3/envs/dsde/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/anaconda3/envs/dsde/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Finished Training


## 이미지를 하나 테스트 해보자

In [11]:
# 사전 훈련된 가중치를 사용하여 GoogLeNet 모델 불러오기
model = googlenet(pretrained=True)

## 이미지 불러오기 및 전처리

In [12]:
from PIL import Image as PILImage # Image 모듈 임포트 추가

In [13]:
# 이미지를 텐서로 변환하고 정규화하는 전처리 함수
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    pil_image = PILImage.open(image_path)
    image = transform(pil_image).unsqueeze(0)  # 배치 차원 추가
    return image

## 이미지 테스트

In [14]:
# 테스트할 이미지 경로 설정
test_image_path = '/Users/hairyungsung/Documents/workspaces/workspace_ds/train_face_0807/기쁨/test_image.jpg'

# 이미지 전처리
input_image = preprocess_image(test_image_path)
# print("전처리된 이미지 텐서 형태:", input_image.shape)



전처리된 이미지 텐서 형태: torch.Size([1, 3, 224, 224])


In [23]:
# 모델에 이미지 전달하여 예측 수행
model.eval()
with torch.no_grad():
    output = model(input_image)

# 예측 결과 확인
_, predicted_class = torch.max(output, 1)
predicted_class_idx = predicted_class.item()

# 클래스 인덱스에 해당하는 클래스 이름 가져오기 (데이터셋 클래스 정보가 필요)
class_names = ['class_기쁨', 'class_당황', 'class_분노', 'class_불안', 'class_상처', 'class_슬픔', 'class_중립']  # 데이터셋의 클래스 정보
predicted_class_name = class_names[predicted_class_idx]

print("Predicted Class:", predicted_class_name)


IndexError: list index out of range

## ---------------아래는 에러를 찾기 위한 정보 확인---------------------

In [21]:
# 클래스 정보 확인
#class_names = train_dataset.classes
#num_classes = len(class_names)

#print("클래스 이름:", class_names)
#print("클래스 수:", num_classes)

In [22]:
# 예측된 클래스 인덱스 출력
print("Predicted Class Index:", predicted_class_idx)

Predicted Class Index: 424


In [16]:
from PIL import Image
import torch
import torchvision.transforms as transforms



# 이미지 전처리 함수
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    pil_image = Image.open(image_path)  # Image 모듈을 PIL로 변경한 변수명 사용
    image = transform(pil_image).unsqueeze(0)  # 배치 차원 추가
    return image

# 테스트할 이미지 경로 설정
test_image_path = '/Users/hairyungsung/Documents/workspaces/workspace_ds/train_face_0807/기쁨/test_image.jpg'

# 이미지 전처리
input_image = preprocess_image(test_image_path)
# print("전처리된 이미지 텐서 형태:", input_image.shape)

# 모델에 이미지 전달하여 예측 수행
model.eval()
with torch.no_grad():
    outputs = model(input_image)
    _, predicted_class_idx = torch.max(outputs, 1)

# 클래스 인덱스에 해당하는 클래스 이름 가져오기 (데이터셋 클래스 정보가 필요)
class_names = ['class_기쁨', 'class_당황', 'class_분노', 'class_불안', 'class_상처', 'class_슬픔', 'class_중립']  # 데이터셋의 클래스 정보
predicted_class_idx = predicted_class_idx.item()  # Tensor를 정수값으로 변환

# 예측된 클래스 인덱스 출력
print("Predicted Class Index:", predicted_class_idx)

# 예측된 클래스 이름 출력
#predicted_class_name = class_names[predicted_class_idx]
#print("Predicted Class Name:", predicted_class_name)


Predicted Class Index: 424


In [17]:
# 클래스 인덱스에 해당하는 클래스 이름 가져오기 (데이터셋 클래스 정보가 필요)
#class_names = ['class_기쁨', 'class_당황', 'class_분노', 'class_불안', 'class_상처', 'class_슬픔', 'class_중립']  # 데이터셋의 클래스 정보

# 예측된 클래스 인덱스를 정수값으로 변환
#predicted_class_idx = int(predicted_class_idx)

# predicted_class_idx 값이 음수일 경우 0으로 조정
#predicted_class_idx = max(predicted_class_idx, 0)

# predicted_class_idx 값이 class_names 리스트의 길이보다 큰 경우 마지막 인덱스로 조정
#predicted_class_idx = min(predicted_class_idx, len(class_names) - 1)

# 예측된 클래스 인덱스 출력
print("Predicted Class Index:", predicted_class_idx)


Predicted Class Index: 424
